In [1]:
import numpy as np
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    LSTM,
    Dense,
    Input,
    Normalization,
    Reshape,
    Flatten,
)
import random
from keras import backend as K
import tensorflow as tf

2024-03-04 13:37:14.030366: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-04 13:37:14.268802: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-04 13:37:14.805118: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-04 13:37:17.954283: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
input_array = np.load(
    "/store/MIAR/training/input_data_180_route.npy",
    allow_pickle=True,
)
print(input_array.shape)

(17156707, 10, 7)


In [12]:
output_array = np.load(
    "/store/MIAR/training/output_data_180_route.npy",
    allow_pickle=True,
)
print(output_array.shape)

(17156707, 180, 3)


In [13]:
lat_mean = np.mean(input_array[:, :, 0])
lat_std = np.std(input_array[:, :, 0])
lon_mean = np.mean(input_array[:, :, 1])
lon_std = np.std(input_array[:, :, 1])
alt_mean = np.mean(input_array[:, :, 2])
alt_std = np.std(input_array[:, :, 2])

In [ ]:
# input_array[:, :, 3] = input_array[:, :, 3].astype("int")
# input_array[:, :, 4] = input_array[:, :, 3].astype("int")
# input_array[:, :, 5] = input_array[:, :, 3].astype("int")
# input_array[:, :, 6] = input_array[:, :, 3].astype("int")

In [ ]:
# input_array = input_array[:, :, 2].reshape(
#     input_array.shape[0], input_array.shape[1], 1
# )
# input_array.shape

# output_array = output_array[:, :, 2].reshape(
#     output_array.shape[0], output_array.shape[1], 1
# )
# output_array.shape

In [14]:
indices = np.arange(input_array.shape[0])
np.random.shuffle(indices)


selected_indices = indices[:50000]
input_subset = input_array[selected_indices]
input_subset_norm = input_subset.copy()
input_subset_norm[:, :, 0] = (input_subset_norm[:, :, 0] - lat_mean) / lat_std
input_subset_norm[:, :, 1] = (input_subset_norm[:, :, 1] - lon_mean) / lon_std
input_subset_norm[:, :, 2] = (input_subset_norm[:, :, 2] - alt_mean) / alt_std


output_subset = output_array[selected_indices]
output_subset_norm = output_subset.copy()
output_subset_norm[:, :, 0] = (
    output_subset_norm[:, :, 0] - lat_mean
) / lat_std
output_subset_norm[:, :, 1] = (
    output_subset_norm[:, :, 1] - lon_mean
) / lon_std
output_subset_norm[:, :, 2] = (
    output_subset_norm[:, :, 2] - alt_mean
) / alt_std

In [15]:
selected_indices = indices[10000:12000]

test_subset_in = input_array[selected_indices]
test_subset_in_norm = test_subset_in.copy()
test_subset_in_norm[:, :, 0] = (
    test_subset_in_norm[:, :, 0] - lat_mean
) / lat_std
test_subset_in_norm[:, :, 1] = (
    test_subset_in_norm[:, :, 1] - lon_mean
) / lon_std
test_subset_in_norm[:, :, 2] = (
    test_subset_in_norm[:, :, 2] - alt_mean
) / alt_std

test_subset_out = output_array[selected_indices]
test_subset_out_norm = test_subset_out.copy()
test_subset_out_norm[:, :, 0] = (
    test_subset_out_norm[:, :, 0] - lat_mean
) / lat_std
test_subset_out_norm[:, :, 1] = (
    test_subset_out_norm[:, :, 1] - lon_mean
) / lon_std
test_subset_out_norm[:, :, 2] = (
    test_subset_out_norm[:, :, 2] - alt_mean
) / alt_std

In [16]:
# Define the input and output sequence length
input_sequence_length = 10  # Observations over 10 timesteps
output_sequence_length = 180  # Predictions over 60 timesteps

# Assuming each timestep has 3 parameters
input_features = 7
output_features = 3

In [ ]:
model = Sequential(
    [
        Input(shape=(input_array.shape[1], input_array.shape[2])),
        # Normalization(),
        # Dense(100, activation="relu"),
        # Dense(100, activation="relu"),
        # LSTM(128, return_sequences=True),
        # LSTM(128, return_sequences=True),
        # LSTM(128, return_sequences=True),
        LSTM(128, return_sequences=True),
        LSTM(128, return_sequences=True),
        LSTM(128),
        # Flatten(),
        # Dense(100, activation="relu"),
        # Dense(100, activation="relu"),
        # Dense(100, activation="relu"),
        # Dense(100, activation="relu"),
        # Dense(100, activation="relu"),
        # Dense(100, activation="relu"),
        # Dense(100, activation="relu"),
        # Dense(100, activation="relu"),
        # Dense(100, activation="relu"),
        # Dense(100, activation="relu"),
        Dense(540, activation="linear"),
        Reshape((output_sequence_length, output_features)),
    ]
)

In [4]:
def rmse_alt(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
    """
    Ca l cul a t e s the RMSE between the t rue and predi c ted Al t i tude values

    Parameters
    ----------
    y_t rue : t f . Tensor
    The t rue Al t i tude values
    y_pred : t f . Tensor
    The predi c ted Al t i tude values

    Returns
    ----------
    t f . Tensor
    RMSE value between the t rue and predi c ted Al t i tude values
    """
    alt_true = tf.gather(y_true, [2], axis=1)
    alt_pred = tf.gather(y_pred, [2], axis=1)
    alt_mean_f = tf.cast((tf.fill([64, 1], alt_mean)), tf.float32)
    alt_std_f = tf.cast((tf.fill([64, 1], alt_std)), tf.float32)
    alt_true_unnorm = tf.math.add((alt_true * alt_std_f), alt_mean_f)
    alt_pred_unnorm = tf.math.add((alt_pred * alt_std_f), alt_mean_f)
    return tf.math.sqrt(
        tf.math.reduce_mean(tf.math.square(alt_pred_unnorm - alt_true_unnorm))
    )


def rmse_lat(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
    """
    Ca l cul a t e s the RMSE between the t rue and predi c ted Al t i tude values

    Parameters
    ----------
    y_t rue : t f . Tensor
    The t rue Al t i tude values
    y_pred : t f . Tensor
    The predi c ted Al t i tude values

    Returns
    ----------
    t f . Tensor
    RMSE value between the t rue and predi c ted Al t i tude values
    """
    lat_true = tf.gather(y_true, [0], axis=1)
    lat_pred = tf.gather(y_pred, [0], axis=1)
    lat_mean_f = tf.cast((tf.fill([64, 1], lat_mean)), tf.float32)
    lat_std_f = tf.cast((tf.fill([64, 1], lat_std)), tf.float32)
    lat_true_unnorm = tf.math.add((lat_true * lat_std_f), lat_mean_f)
    lat_pred_unnorm = tf.math.add((lat_pred * lat_std_f), lat_mean_f)
    return tf.math.sqrt(
        tf.math.reduce_mean(tf.math.square(lat_pred_unnorm - lat_true_unnorm))
    )


def rmse_lon(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
    """
    Ca l cul a t e s the RMSE between the t rue and predi c ted Al t i tude values

    Parameters
    ----------
    y_t rue : t f . Tensor
    The t rue Al t i tude values
    y_pred : t f . Tensor
    The predi c ted Al t i tude values

    Returns
    ----------
    t f . Tensor
    RMSE value between the t rue and predi c ted Al t i tude values
    """
    lon_true = tf.gather(y_true, [1], axis=1)
    lon_pred = tf.gather(y_pred, [1], axis=1)
    lon_mean_f = tf.cast((tf.fill([64, 1], lon_mean)), tf.float32)
    lon_std_f = tf.cast((tf.fill([64, 1], lon_std)), tf.float32)
    lon_true_unnorm = tf.math.add((lon_true * lon_std_f), lon_mean_f)
    lon_pred_unnorm = tf.math.add((lon_pred * lon_std_f), lon_mean_f)
    return tf.math.sqrt(
        tf.math.reduce_mean(tf.math.square(lon_pred_unnorm - lon_true_unnorm))
    )

In [ ]:
model.compile(
    # loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    # optimizer=keras.optimizers.RMSprop(),
    # metrics=["accuracy"],
    loss="mean_squared_error",
    optimizer="adam",
    metrics=[rmse_lat, rmse_lon, rmse_alt],
)

In [ ]:
# Model summary to understand the architecture
model.summary()

In [ ]:
model.fit(
    input_subset_norm,
    output_subset_norm,
    epochs=500,
    batch_size=64,
    validation_split=0.2,
)

In [ ]:
model.save("/home/krum/git/MT_krum_code/src/02_model_training/model.keras")

In [7]:
from tensorflow.keras.models import load_model

model = load_model(
    "/home/krum/git/MT_krum_code/src/02_model_training/model.keras",
    custom_objects={
        "rmse_lat": rmse_lat,
        "rmse_lon": rmse_lon,
        "rmse_alt": rmse_alt,
    },
)

In [17]:
prediction = model.predict(test_subset_in_norm)

63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step


In [18]:
prediction[:, :, 0] = prediction[:, :, 0] * lat_std + lat_mean
prediction[:, :, 1] = prediction[:, :, 1] * lon_std + lon_mean
prediction[:, :, 2] = prediction[:, :, 2] * alt_std + alt_mean

In [232]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

x = random.randint(0, prediction.shape[0])

# Generate subplots------------------------------------------------------------
fig = make_subplots(
    rows=2,
    cols=1,
    specs=[[{"type": "scattermapbox"}], [{}]],
    row_heights=[0.66, 0.33],
    subplot_titles=("Position", "Altitude"),
    vertical_spacing=0.05,
)

# Add traces to the first plot (Position)--------------------------------------
# Input
fig.append_trace(
    go.Scattermapbox(
        mode="lines",
        lat=test_subset_in[x, :, 0].reshape(10),
        lon=test_subset_in[x, :, 1].reshape(10),
        line=dict(width=2, color="#636efa"),
        name="input",
    ),
    row=1,
    col=1,
)

# Actual
fig.append_trace(
    go.Scattermapbox(
        mode="lines",
        lat=test_subset_out[x, :, 0].reshape(180),
        lon=test_subset_out[x, :, 1].reshape(180),
        line=dict(width=2, color="#ef553b"),
        name="actual",
    ),
    row=1,
    col=1,
)

# Predicted
fig.append_trace(
    go.Scattermapbox(
        mode="lines",
        lat=prediction[x, :, 0].reshape(180),
        lon=prediction[x, :, 1].reshape(180),
        line=dict(width=2, color="#00cc96"),
        name="predicted",
    ),
    row=1,
    col=1,
)

# Add traces to the second plot (Altitude)-------------------------------------
fig.append_trace(
    go.Scatter(
        x=np.arange(0, 10),
        y=test_subset_in[x, :, 2].reshape(10),
        line=dict(width=2, color="#636efa"),
        mode="lines",
        name="input",
        showlegend=False,
    ),
    row=2,
    col=1,
)

fig.append_trace(
    go.Scatter(
        x=np.arange(10, 190),
        y=test_subset_out[x, :, 2].reshape(180),
        line=dict(width=2, color="#ef553b"),
        mode="lines",
        name="actual",
        showlegend=False,
    ),
    row=2,
    col=1,
)
fig.append_trace(
    go.Scatter(
        x=np.arange(10, 190),
        y=prediction[x, :, 2].reshape(180),
        line=dict(width=2, color="#00cc96"),
        mode="lines",
        name="prediction",
        showlegend=False,
    ),
    row=2,
    col=1,
)

fig.update_xaxes(title_text="Elapsed time [s]", row=2, col=1)
fig.update_yaxes(title_text="Baroaltitude [ft]", row=2, col=1)

# --------------------------------------------------------------------------
fig.update_layout(
    mapbox=dict(
        style="carto-positron",
        zoom=9,
        center=dict(
            lat=np.mean(test_subset_in[x, :, 0].reshape(10)),
            lon=np.mean(test_subset_in[x, :, 1].reshape(10)),
        ),
    ),
)

fig.update_layout(
    width=1200,
    height=1000,
    margin=dict(l=50, r=0, t=40, b=40),
)

fig.show()